## zillow data

In [191]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [210]:
path = '/Users/sanjayagrawal/Downloads/Kaggle/zillow/'
sample_submission = pd.read_csv(path + 'sample_submission.csv')
train_2016_v2 = pd.read_csv(path + 'train_2016_v2.csv')
properties_2016 = pd.read_csv(path + 'properties_2016.csv')

/Users/sanjayagrawal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [211]:
for i in properties_2016.columns:
    if properties_2016.dtypes[i] == 'object':
        print (i)
        properties_2016[i] = properties_2016[i].map(lambda x : str(x))
        le = preprocessing.LabelEncoder()
        properties_2016[i] = le.fit_transform(properties_2016[i])

hashottuborspa
propertycountylandusecode
propertyzoningdesc
fireplaceflag
taxdelinquencyflag


In [212]:
train_2016_v2 = train_2016_v2.rename(columns = {'parcelid':'ParcelId'})
properties_2016 = properties_2016.rename(columns = {'parcelid':'ParcelId'})
train = train_2016_v2.merge(properties_2016, on='ParcelId', how = 'inner')
test = sample_submission.merge(properties_2016, on='ParcelId', how = 'inner')

In [213]:
# for i in train.columns:
#     if train[i].isnull().sum() > 50000:
#         train = train.drop(i, axis=1)
#         test = test.drop(i, axis=1)
        
train = train.fillna(-999)
test = test.fillna(-999)

## Data Split with sklearn

In [327]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
x = train.drop(['ParcelId', 'logerror', 'transactiondate'], axis=1)
y= train['logerror']

x_train, y_train, x_vald, y_vald = train_test_split(x, y, train_size=0.80, random_state=1234)

x_test = test.drop(['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712'], axis=1)
print (x.shape, x_test.shape)

(90275, 57) (2985217, 0)


/Users/sanjayagrawal/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## LightGBM model

In [336]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 40,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.6,
    'bagging_freq': 2,
    'max_depth' : -1,
    'max_bin' : 32,
    'verbose': 0
}

In [330]:
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.03],
    'n_estimators': [40],
    'feature_fraction': [0.9],
    'task': ['train'],
    'num_leaves' : [40],
    'boosting_type': ['gbdt'],
    'objective': ['regression'],
    'metric': [{'l2', 'auc'}],
    'bagging_fraction': [0.6],
    'bagging_freq': [2],
    'max_depth' : [-1],
    'max_bin' : [32],
    'verbose': [0]
}

gbm = GridSearchCV(estimator, param_grid)
gbm.fit(x, y)
print('Best parameters found by grid search are:', gbm.best_params_)

In [ ]:
(boosting_type=’gbdt’, num_leaves=31, max_depth=-1, learning_rate=0.1,
n_estimators=10, max_bin=255, subsample_for_bin=50000,
objective=None, min_split_gain=0,
min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
colsample_bytree=1, reg_alpha=0, reg_lambda=0,
random_state=0, n_jobs=-1, silent=True, **kwargs)

In [341]:
lgb_train = lgb.Dataset(x_train, x_vald)
lgb_eval = lgb.Dataset(y_train, y_vald, reference=lgb_train)
gbm = lgb.train(params, lgb_train, num_boost_round=500, valid_sets=lgb_eval, early_stopping_rounds=100)

[1]	valid_0's l2: 0.0247161	valid_0's auc: 0.533086
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l2: 0.0247038	valid_0's auc: 0.534779
[3]	valid_0's l2: 0.0246867	valid_0's auc: 0.535393
[4]	valid_0's l2: 0.0246714	valid_0's auc: 0.538159
[5]	valid_0's l2: 0.0246528	valid_0's auc: 0.549253
[6]	valid_0's l2: 0.0246366	valid_0's auc: 0.553866
[7]	valid_0's l2: 0.0246206	valid_0's auc: 0.55402
[8]	valid_0's l2: 0.024609	valid_0's auc: 0.555034
[9]	valid_0's l2: 0.024597	valid_0's auc: 0.556266
[10]	valid_0's l2: 0.0245892	valid_0's auc: 0.556195
[11]	valid_0's l2: 0.0245725	valid_0's auc: 0.558585
[12]	valid_0's l2: 0.0245565	valid_0's auc: 0.559095
[13]	valid_0's l2: 0.0245506	valid_0's auc: 0.559434
[14]	valid_0's l2: 0.0245481	valid_0's auc: 0.559104
[15]	valid_0's l2: 0.02454	valid_0's auc: 0.558936
[16]	valid_0's l2: 0.0245291	valid_0's auc: 0.559475
[17]	valid_0's l2: 0.0245215	valid_0's auc: 0.560385
[18]	valid_0's l2: 0.0245147	valid_0's auc: 0.5610

[162]	valid_0's l2: 0.0244218	valid_0's auc: 0.569458
[163]	valid_0's l2: 0.0244224	valid_0's auc: 0.569409
[164]	valid_0's l2: 0.0244274	valid_0's auc: 0.569186
[165]	valid_0's l2: 0.0244254	valid_0's auc: 0.56916
[166]	valid_0's l2: 0.0244244	valid_0's auc: 0.569255
[167]	valid_0's l2: 0.0244225	valid_0's auc: 0.569284
[168]	valid_0's l2: 0.0244209	valid_0's auc: 0.569257
[169]	valid_0's l2: 0.0244211	valid_0's auc: 0.569455
[170]	valid_0's l2: 0.0244231	valid_0's auc: 0.569413
[171]	valid_0's l2: 0.0244239	valid_0's auc: 0.569359
[172]	valid_0's l2: 0.0244268	valid_0's auc: 0.569404
[173]	valid_0's l2: 0.0244292	valid_0's auc: 0.569373
[174]	valid_0's l2: 0.0244318	valid_0's auc: 0.569494
[175]	valid_0's l2: 0.0244276	valid_0's auc: 0.569561
[176]	valid_0's l2: 0.0244267	valid_0's auc: 0.569704
[177]	valid_0's l2: 0.0244263	valid_0's auc: 0.569593
[178]	valid_0's l2: 0.0244262	valid_0's auc: 0.56984
[179]	valid_0's l2: 0.024427	valid_0's auc: 0.56969
[180]	valid_0's l2: 0.0244273	va

In [233]:
# categorical_feature=[list(x).index(catFeature) for catFeature in x]
# categorical_feature

## Predictions

In [342]:
# gbm.save_model('model.txt')
y_pred = gbm.predict(x_test)
test['pred'] = y_pred

In [343]:
test['201610'] = test['pred'].map(lambda x : x)
test['201611'] = test['pred'].map(lambda x : x)
test['201612'] = test['pred'].map(lambda x : x)
test['201710'] = test['pred'].map(lambda x : x)
test['201711'] = test['pred'].map(lambda x : x)
test['201712'] = test['pred'].map(lambda x : x)
test = test[['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']]

In [344]:
test.to_csv('./final10.csv', index=False)

In [ ]:
test.head()